In [ ]:
from src.env_host.cloud.cloud_launcher import CloudEnvLauncher 
from src.env_host.launcher import EnvLauncher
from src.config.aws_config import EC2Config
 
env_id = "3DBallHard"
# 1) Acquire a launcher object
cloud_env_launcher: CloudEnvLauncher = EnvLauncher.launch_on_cloud(env_simulator="unity", env_id=env_id, env_file_path="../unity_environments/3DBallHard", 
                                                                   global_image_name="cloud_" + env_id.lower(), ecr_registry="123456789012.dkr.ecr.us-east-1.amazonaws.com",
                                                                   ec2_config=EC2Config(instance_type="g5.xlarge", ensure_ami_config=True))

In [ ]:

# 2) Generate or prepare a Docker folder
dockerfile_path = cloud_env_launcher.generate_dockerfile(copy_env_file_if_outside = True)


In [ ]:
# 3) Build the Docker image from that folder
cloud_env_launcher.build_docker_image(dockerfile_path=dockerfile_path)

In [ ]:

# 4) Push the Docker image to a registry (e.g., ECR)
cloud_env_launcher.push_docker_image(ensure_ecr_login=True, ensure_ecr_repo=True)

In [ ]:

# 5) Launch an EC2 instance, passing user_data
cloud_env_launcher.launch_ec2_instance()

In [ ]:
# 6) Retrieve the endpoint
env_endpoint = cloud_env_launcher.get_env_endpoint()
# env_endpoint = "http://ec2-3-235-11-111.compute-1.amazonaws.com"

env_endpoint = EnvLauncher.launch_on_local_with_ip(env_simulator='unity', ip_address="")

env_endpoint = EnvLauncher.launch_on_local_with_url(env_simulator='unity', tunnel_name="ngrok")

In [ ]:
# main.ipynb train side
from src.agent_gpt import AgentGPT
from src.config.hyperparams import Hyperparameters
from src.config.aws_config import SageMakerConfig
role_arn = "arn:aws:iam::123456789012:role/SageMakerExecutionRole"
image_uri = "123456789012.dkr.ecr.us-east-1.amazonaws.com/agent-gpt-trainer:latest"
output_path="s3://agentgpt-test/"
sagemaker_config=SageMakerConfig(output_path = output_path, image_uri = image_uri, role_arn = role_arn)

In [ ]:
sagemaker_config.model_data = "s3://your_agentgpt/model.tar.gz"
AgentGPT.train_on_cloud(sagemaker_config, hyperparameters=Hyperparameters(env_id, env_endpoint))

In [ ]:
agent_gpt = AgentGPT.run_on_cloud(sagemaker_config)

In [ ]:
import numpy as np

# ---------------------------------------------------------------------
# 1) Test set_control_value / get_control_value
# ---------------------------------------------------------------------
agent_gpt.set_control_value(2.0)
current_control = agent_gpt.get_control_value()
print(f"[TEST] control_value set to: {current_control}")

# ---------------------------------------------------------------------
# 2) Test sample_observation / sample_action
# ---------------------------------------------------------------------
obs = agent_gpt.sample_observation()
print(f"[TEST] Sampled observation shape: {obs.shape}")
act = agent_gpt.sample_action()
print(f"[TEST] Sampled action: {act}")

# ---------------------------------------------------------------------
# 3) Test select_action
#    (Here we simulate 2 agents (IDs=0,1) each receiving the same obs)
# ---------------------------------------------------------------------
actions = agent_gpt.select_action(
    agent_ids=[0, 1],
    observations=[obs, obs],
)
print(f"[TEST] Actions returned by select_action:\n{actions}")

# ---------------------------------------------------------------------
# 4) Test remove_agents
#    We remove agent "0" from the system.
# ---------------------------------------------------------------------
agent_gpt.terminate_agents(np.array([0]))
print(f"[TEST] After removing agent 0, status:\n{agent_gpt.status()}")

# ---------------------------------------------------------------------
# 5) Test reset_agents
#    Let's reset the entire agent registry (if you want to clear all).
# ---------------------------------------------------------------------
agent_gpt.reset_agents(5)  # or however many you want to reset
print(f"[TEST] After resetting, status:\n{agent_gpt.status()}")

# ---------------------------------------------------------------------
# 6) Final status check
# ---------------------------------------------------------------------
final_status = agent_gpt.status()
print(f"[TEST] Final Status:\n{final_status}")
